# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF


In [4]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## Question 1

This will hold true regardless of whether the source population is normal or skewed, provided the sample size is sufficiently large (usually n > 30). If the population is normal, then the theorem holds true even for samples smaller than 30. In fact, this also holds true even if the population is binomial, provided that min(np, n(1-p))> 5, where n is the sample size and p is the probability of success in the population. This means that we can use the normal probability model to quantify uncertainty when making inferences about a population mean based on the sample mean. (http://sphweb.bumc.bu.edu/otlt/mph-modules/bs/bs704_probability/BS704_Probability12.html)

In [26]:
# First of all separate the resumes by race: b and w.
df_white=data[data.race=='w']
df_black=data[data.race=='b']

#Number of CV per race:
w_cvs=len(df_white.race)
b_cvs=len(df_black.race)

#Number of calls per race:
w_calls=sum(data[data.race=='w'].call)
b_calls=sum(data[data.race=='b'].call)

# Sample proportions:
w_sample_p = w_calls / w_cvs
b_sample_p = b_calls / b_cvs

print(w_sample_p,b_sample_p)

0.0965092402464 0.064476386037


In [27]:
ppool= round((w_calls+b_calls)/(w_cvs+b_cvs),2)
ppool

0.080000000000000002

In [28]:

# Then let's check if n1p_pool >=10, n1(1-p_pool) >=10:

np_w=w_cvs*ppool
n1p_w=w_cvs*(1-ppool)

# and for n2p_pool >=10, n2(1-p_pool) >=10:
np_b=b_cvs*ppool
n1p_b=b_cvs*(1-ppool)

print(np_w,n1p_w, np_b,n1p_b)

194.8 2240.2 194.8 2240.2


Since the ^ values are above 10 they meet the conditions for CLT, but I don't even think we needed to do this since the sample size was above 30.

### Question 2: What is the Null and Alternative Hypothesis?

Null: The null hypothesis is the commonly accepted hypothesis. In this situation the null hypothesis would be that the sound of a name has no affect on whether or not they will receive a call. So calls will be distributed 50% to white and 50% to black

Alternative: The alternative hypothesis is the idea of trying to prove the null hypothesis wrong. In this case the alternative hypothesis is that the sound of a name does affect whether or not they will receive a call.

### Question 3: Margin of error, confidence interval, and p-value

In [29]:

# To calculate SE we use the null hypothesis as true. 
# It means that mean = 0, and SE uses p_pool as population proportion:

SE_diff=round((((ppool*(1-ppool))/b_cvs)+((ppool*(1-ppool))/w_cvs))**0.5,2)
SE_diff

0.01

In [31]:
#z-value
Zvalue= round((w_sample_p - b_sample_p)/SE_diff,2)
#print Zvalue

# ME 
ME=round(Zvalue*SE_diff,2)
print(ME)

0.03


In [32]:

#  CI = (estimate point + ME, estimate point - ME)

Up_CI= (w_sample_p-b_sample_p)+ME
Down_CI= (w_sample_p-b_sample_p)-ME

print(" The CONFIDENT INTERVAL IS: (",Down_CI, ",", Up_CI,")")


 The CONFIDENT INTERVAL IS: ( 0.00203285420945 , 0.0620328542094 )


In [ ]:

# Z-score= 3.2
# using r with the function: pnorm(3.2,lower.tail=FALSE)*2
# the result is: 0.001374276

### Discuss statistical significance
The low value of the p-value indicates that we can reject the null hypothesis. It means that race does have an affect on how many resumes are called.